# Session 1: Introduction to Gaussian Processes and PyMC

**Duration:** 3 hours  
**Workshop:** Gaussian Processes with PyMC and LLMs

---

Gaussian Processes (GPs) represent one of the most elegant and powerful tools in modern machine learning and statistics. Unlike parametric models that assume specific functional forms, GPs provide a **non-parametric approach** that can capture complex patterns while naturally quantifying uncertainty. This makes them particularly valuable for applications where understanding uncertainty is as important as making predictions—from scientific modeling to decision-making under uncertainty.

This session introduces the foundational concepts of Gaussian Processes within the PyMC probabilistic programming framework. We will build intuition about what it means for a process to be "Gaussian," explore the mathematical machinery that makes GPs work, and learn to implement them using PyMC's powerful and expressive interface.

## Why Gaussian Processes?

Traditional machine learning often focuses on finding the "best" parameters for a pre-specified model. Gaussian Processes take a fundamentally different approach: instead of assuming a specific functional form, they place a probability distribution directly over the **space of functions**. This perspective offers several compelling advantages:

- **Principled uncertainty quantification**: GPs provide natural confidence intervals and probability distributions over predictions
- **Automatic model selection**: Through marginal likelihood optimization, GPs can automatically tune their complexity to the data
- **Incorporation of prior knowledge**: Domain expertise can be encoded through choice of mean functions and covariance kernels
- **Small data efficiency**: GPs can make meaningful predictions and quantify uncertainty even with limited training data
- **Interpretable hyperparameters**: Kernel parameters often have clear physical or domain-specific meanings

## Learning Objectives

By the end of this session, you will be able to:

1. **Understand the mathematical foundations of GPs**: Grasp how Gaussian Processes extend multivariate Gaussian distributions to infinite-dimensional function spaces
2. **Build intuition through visualization**: Create and interpret samples from GP priors to understand how hyperparameters affect function behavior
3. **Master PyMC's probabilistic programming paradigm**: Use PyMC's model contexts, distributions, and inference machinery for GP modeling
4. **Construct and analyze covariance functions**: Build kernels from first principles and understand their role in encoding assumptions about function smoothness and structure
5. **Navigate PyMC's GP implementations**: Understand the trade-offs between `gp.Marginal` and `gp.Latent` approaches and when to use each
6. **Apply GPs to real problems**: Build complete GP regression models, from prior specification through posterior inference to prediction

## Session Structure

This session is organized into six major sections, each building on the previous one:

1. **Mathematical Foundations** (45 minutes): Core concepts, definitions, and the connection between finite and infinite-dimensional Gaussians
2. **PyMC Fundamentals** (45 minutes): Model contexts, distributions, random variables, and the probabilistic programming paradigm
3. **Kernel Theory and Construction** (45 minutes): Understanding covariance functions as the heart of GP modeling
4. **PyMC GP Implementations** (45 minutes): Comparing marginal vs. latent formulations with practical examples
5. **Hands-on Practice** (30 minutes): Guided exercises to reinforce key concepts
6. **Integration and Next Steps** (15 minutes): Synthesis and preview of advanced topics

Let's begin our journey into the world of Gaussian Processes and probabilistic programming.

---

## Environment Setup

We begin by setting up our computational environment with the necessary libraries for Gaussian Process modeling, Bayesian inference, and visualization. This section establishes the foundation for all subsequent analysis.

In [ ]:
# Core scientific computing
import numpy as np
import scipy.stats as stats
import polars as pl

# PyMC ecosystem for probabilistic programming
import pymc as pm
import pytensor.tensor as pt
import arviz as az

# Visualization libraries
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# Configure visualization defaults
az.style.use('arviz-doc')
pio.templates.default = 'plotly_white'
px.defaults.template = 'plotly_white'
px.defaults.width = 800
px.defaults.height = 500

# Reproducibility
RANDOM_SEED = 20090425
RNG = np.random.default_rng(RANDOM_SEED)

print(f"Environment configured successfully!")
print(f"PyMC version: {pm.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Random seed: {RANDOM_SEED}")

---

## Part I: Mathematical Foundations of Gaussian Processes

To understand Gaussian Processes deeply, we must first establish their mathematical foundations. This section will build intuition by connecting familiar concepts (univariate and multivariate Gaussians) to the more abstract notion of distributions over functions.

### From Scalars to Functions: The Gaussian Hierarchy

The conceptual progression from simple to complex Gaussian structures provides the key to understanding GPs:

1. **Univariate Gaussian**: $X \sim \mathcal{N}(\mu, \sigma^2)$ describes uncertainty about a single scalar value
2. **Multivariate Gaussian**: $\mathbf{X} \sim \mathcal{N}(\boldsymbol{\mu}, \Sigma)$ describes uncertainty about a finite-dimensional vector
3. **Gaussian Process**: $f(\cdot) \sim \mathcal{GP}(m(\cdot), k(\cdot, \cdot))$ describes uncertainty about an infinite-dimensional function

The remarkable insight of Gaussian Processes is that we can work with infinite-dimensional function spaces by considering only finite-dimensional marginals at any collection of input points.

### Formal Definition

**Definition**: A Gaussian Process is a collection of random variables, any finite number of which have a joint Gaussian distribution.

More precisely, a stochastic process $\{f(x) : x \in \mathcal{X}\}$ is a Gaussian Process if for any finite set of indices $\{x_1, x_2, \ldots, x_n\} \subset \mathcal{X}$, the joint distribution of the random vector $(f(x_1), f(x_2), \ldots, f(x_n))^T$ is multivariate Gaussian.

A Gaussian Process is completely specified by two functions:

1. **Mean function**: $m(x) = \mathbb{E}[f(x)]$
2. **Covariance function**: $k(x, x') = \mathbb{Cov}[f(x), f(x')] = \mathbb{E}[(f(x) - m(x))(f(x') - m(x'))]$

We denote this as:
$$f(x) \sim \mathcal{GP}(m(x), k(x, x'))$$

### The Finite-Dimensional View

For any finite collection of input points $\mathbf{X} = \{x_1, x_2, \ldots, x_n\}$, the corresponding function values $\mathbf{f} = [f(x_1), f(x_2), \ldots, f(x_n)]^T$ follow a multivariate Gaussian distribution:

$$\mathbf{f} \sim \mathcal{N}(\boldsymbol{\mu}, \mathbf{K})$$

where:
- $\boldsymbol{\mu} = [m(x_1), m(x_2), \ldots, m(x_n)]^T$ is the mean vector
- $\mathbf{K}$ is the covariance matrix with entries $K_{ij} = k(x_i, x_j)$

This finite-dimensional perspective is crucial because it allows us to:
- Sample functions from the GP (by sampling from the multivariate Gaussian)
- Compute likelihoods (using the multivariate Gaussian density)
- Perform inference (using standard multivariate Gaussian conditioning)

### Properties of Covariance Functions

The covariance function $k(x, x')$ is the heart of a Gaussian Process. It encodes our assumptions about function smoothness, periodicity, and other structural properties. For $k$ to be a valid covariance function, it must be:

1. **Symmetric**: $k(x, x') = k(x', x)$ for all $x, x'$
2. **Positive semi-definite**: For any finite set $\{x_1, \ldots, x_n\}$, the matrix $\mathbf{K}$ with $K_{ij} = k(x_i, x_j)$ must be positive semi-definite

These conditions ensure that the resulting covariance matrices are valid, guaranteeing that we can sample from and compute probabilities under the GP.

### Building Intuition: From Multivariate Gaussian to GP

Let's build intuition by starting with a simple multivariate Gaussian and then extending to the GP setting. We'll see how increasing the number of dimensions naturally leads us to the function space perspective.

In [ ]:
def create_rbf_covariance(X, length_scale=1.0, variance=1.0):
    """
    Create RBF (Radial Basis Function) covariance matrix.
    
    The RBF kernel is defined as:
    k(x, x') = σ² * exp(-||x - x'||² / (2ℓ²))
    
    Parameters:
    -----------
    X : array-like, shape (n,)
        Input locations
    length_scale : float
        Length scale parameter ℓ
    variance : float  
        Variance parameter σ²
        
    Returns:
    --------
    K : ndarray, shape (n, n)
        Covariance matrix
    """
    X = np.asarray(X).reshape(-1, 1) if np.asarray(X).ndim == 1 else np.asarray(X)
    
    # Compute squared Euclidean distances
    sqdist = np.sum(X**2, axis=1)[:, None] + np.sum(X**2, axis=1)[None, :] - 2 * np.dot(X, X.T)
    
    # RBF covariance
    K = variance * np.exp(-0.5 * sqdist / length_scale**2)
    
    return K

def zero_mean_function(X):
    """Zero mean function."""
    return np.zeros(len(X))

# Demonstrate the progression from few to many points
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=["5 points", "10 points", "25 points", "50 points"],
                    vertical_spacing=0.1)

n_points_list = [5, 10, 25, 50]
colors = ['blue', 'red', 'green', 'orange']

for idx, (n_points, color) in enumerate(zip(n_points_list, colors)):
    # Create input points
    X = np.linspace(-3, 3, n_points)
    
    # Create covariance matrix
    K = create_rbf_covariance(X, length_scale=1.0, variance=1.0)
    
    # Add small jitter for numerical stability
    K += 1e-6 * np.eye(len(X))
    
    # Sample functions
    mu = zero_mean_function(X)
    f_samples = RNG.multivariate_normal(mu, K, size=3)
    
    # Plot settings
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    # Plot samples
    for i, f in enumerate(f_samples):
        fig.add_trace(
            go.Scatter(x=X, y=f, mode='lines+markers', 
                      line=dict(color=color, width=2),
                      marker=dict(size=4),
                      name=f"Sample {i+1}" if idx == 0 else None,
                      showlegend=idx == 0,
                      opacity=0.7),
            row=row, col=col
        )
    
    # Plot mean and confidence bands
    std = np.sqrt(np.diag(K))
    fig.add_trace(
        go.Scatter(x=np.concatenate([X, X[::-1]]),
                  y=np.concatenate([mu + 2*std, (mu - 2*std)[::-1]]),
                  fill='toself', fillcolor='rgba(128,128,128,0.2)',
                  line=dict(color='rgba(255,255,255,0)'),
                  name="±2σ" if idx == 0 else None,
                  showlegend=idx == 0),
        row=row, col=col
    )
    
    fig.add_trace(
        go.Scatter(x=X, y=mu, mode='lines',
                  line=dict(color='black', width=2, dash='dash'),
                  name="Mean" if idx == 0 else None,
                  showlegend=idx == 0),
        row=row, col=col
    )

fig.update_layout(
    height=600,
    title_text="Progression from Multivariate Gaussian to Gaussian Process",
    showlegend=True
)

fig.update_xaxes(title_text="Input x")
fig.update_yaxes(title_text="Function value f(x)")

fig.show()

print("As we increase the number of points, we approach a continuous function sampled from a GP.")
print("Each subplot shows 3 different function samples from the same GP prior.")

**Key Insight**: As we increase the number of evaluation points, the discrete samples begin to resemble continuous functions. In the limit, we have a Gaussian Process that defines a probability distribution over the entire function space.

### Understanding Covariance Matrices

The covariance matrix $\mathbf{K}$ encodes all the structural assumptions we make about our functions. Let's visualize how different hyperparameters affect the covariance structure:

In [ ]:
# Create a small set of points to visualize covariance matrices
X_small = np.linspace(0, 4, 5)

# Different hyperparameter configurations
configs = [
    {'length_scale': 0.5, 'variance': 1.0, 'title': 'Short Length Scale (ℓ=0.5)'},
    {'length_scale': 2.0, 'variance': 1.0, 'title': 'Long Length Scale (ℓ=2.0)'},
    {'length_scale': 1.0, 'variance': 0.5, 'title': 'Low Variance (σ²=0.5)'},
    {'length_scale': 1.0, 'variance': 2.0, 'title': 'High Variance (σ²=2.0)'}
]

fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=[config['title'] for config in configs],
                    vertical_spacing=0.15)

for idx, config in enumerate(configs):
    # Create covariance matrix
    K = create_rbf_covariance(X_small, 
                             length_scale=config['length_scale'],
                             variance=config['variance'])
    
    # Plot settings
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    # Create heatmap
    fig.add_trace(
        go.Heatmap(z=K, 
                   x=[f"{x:.1f}" for x in X_small],
                   y=[f"{x:.1f}" for x in X_small],
                   colorscale='Viridis',
                   showscale=idx == 0,  # Only show colorscale for first plot
                   text=np.round(K, 3),
                   texttemplate="%{text}",
                   textfont={"size": 10}),
        row=row, col=col
    )

fig.update_layout(
    height=600,
    title_text="Covariance Matrices with Different Hyperparameters"
)

fig.show()

print("Covariance Matrix Interpretation:")
print("• Diagonal elements: Variance at each point (should equal σ²)")
print("• Off-diagonal elements: Covariance between different points")
print("• Length scale controls how quickly covariance decays with distance")
print("• Variance parameter scales the overall magnitude of the covariance")

---

## Part II: PyMC Fundamentals for Probabilistic Programming

Before diving into Gaussian Processes specifically, we need to understand PyMC's approach to probabilistic programming. PyMC provides a powerful framework for specifying, fitting, and analyzing Bayesian models through an intuitive Python interface.

### The Philosophy of Probabilistic Programming

Probabilistic programming represents a paradigm shift in statistical modeling. Instead of deriving update equations or coding samplers by hand, we declare the structure of our model and let the framework handle the computational details. This approach offers several advantages:

- **Model specification mirrors mathematical notation**: Code looks like the mathematical model
- **Automatic inference**: No need to implement custom sampling algorithms
- **Composability**: Complex models can be built from simpler components
- **Flexibility**: Easy to experiment with different model structures

### PyMC's Core Components

PyMC organizes probabilistic models around several key abstractions:

1. **Model Context**: A context manager that tracks all model components
2. **Random Variables**: Represent uncertain quantities with probability distributions
3. **Deterministic Variables**: Represent quantities that are functions of other variables
4. **Observed Variables**: Represent data that we condition on

Let's explore each of these concepts through examples.

### Model Contexts and Random Variables

Every PyMC model exists within a **Model context**. This context manager keeps track of all model components and their relationships:

In [ ]:
# Create a simple model context
with pm.Model() as simple_model:
    # Define a random variable
    theta = pm.Normal('theta', mu=0, sigma=1)
    
    # The model automatically tracks this variable
    print(f"Model variables: {list(simple_model.named_vars.keys())}")
    print(f"Variable type: {type(theta)}")
    
# We can examine the model structure
print(f"\nModel summary:")
print(f"Number of free random variables: {len(simple_model.free_RVs)}")
print(f"Number of observed variables: {len(simple_model.observed_RVs)}")

### Working with Distributions

PyMC provides a comprehensive library of probability distributions. Let's explore some commonly used distributions and their properties:

In [ ]:
# Demonstrate different distribution types
with pm.Model() as distribution_demo:
    
    # Continuous distributions
    normal_var = pm.Normal('normal', mu=0, sigma=1)
    gamma_var = pm.Gamma('gamma', alpha=2, beta=1)
    beta_var = pm.Beta('beta', alpha=2, beta=2)
    
    # Discrete distributions  
    binomial_var = pm.Binomial('binomial', n=10, p=0.3)
    poisson_var = pm.Poisson('poisson', mu=3)
    
    # Half-distributions (positive support)
    half_normal_var = pm.HalfNormal('half_normal', sigma=1)
    
    print("Distribution types in the model:")
    for var_name, var in distribution_demo.named_vars.items():
        print(f"  {var_name}: {var.owner.op.__class__.__name__}")

### Sampling from Distributions

PyMC provides several ways to sample from distributions. The `pm.draw()` function allows us to sample from the prior distributions:

In [ ]:
# Sample from distributions
with distribution_demo:
    # Sample single values
    print("Single samples:")
    print(f"Normal: {pm.draw(normal_var):.3f}")
    print(f"Gamma: {pm.draw(gamma_var):.3f}")
    print(f"Beta: {pm.draw(beta_var):.3f}")
    print(f"Binomial: {pm.draw(binomial_var)}")
    print(f"Poisson: {pm.draw(poisson_var)}")
    
    # Sample multiple values
    normal_samples = pm.draw(normal_var, draws=1000)
    print(f"\n1000 Normal samples - Mean: {normal_samples.mean():.3f}, Std: {normal_samples.std():.3f}")

### Computing Log-Probabilities

A fundamental operation in Bayesian inference is computing log-probabilities. PyMC provides the `pm.logp()` function for this purpose:

In [ ]:
# Compute log-probabilities
with distribution_demo:
    # Evaluate log-probability at specific values
    print("Log-probabilities:")
    print(f"Normal(0) at x=0: {pm.logp(normal_var, 0).eval():.3f}")
    print(f"Normal(0) at x=2: {pm.logp(normal_var, 2).eval():.3f}")
    print(f"Gamma(α=2,β=1) at x=1: {pm.logp(gamma_var, 1).eval():.3f}")
    print(f"Beta(α=2,β=2) at x=0.5: {pm.logp(beta_var, 0.5).eval():.3f}")
    
    # Compare to scipy for verification
    scipy_normal_logpdf = stats.norm.logpdf(0, loc=0, scale=1)
    pymc_normal_logp = pm.logp(normal_var, 0).eval()
    print(f"\nVerification - SciPy: {scipy_normal_logpdf:.6f}, PyMC: {pymc_normal_logp:.6f}")

### Deterministic Variables and Transformations

Often we need to create variables that are deterministic functions of other variables. PyMC provides two approaches: anonymous transformations and named deterministic variables:

In [ ]:
with pm.Model() as transformation_model:
    # Base random variables
    x = pm.Normal('x', mu=0, sigma=1)
    y = pm.Normal('y', mu=0, sigma=1)
    
    # Anonymous transformation (not tracked in output)
    z_anonymous = x + y  # This won't appear in sampling output
    
    # Named deterministic (tracked in output)
    z_named = pm.Deterministic('sum_xy', x + y)
    squared = pm.Deterministic('x_squared', x**2)
    
    # We can also create more complex transformations
    complex_transform = pm.Deterministic('complex', 
                                       pt.sin(x) * pt.exp(y / 2))

print("Variables in transformation model:")
for name in transformation_model.named_vars.keys():
    print(f"  {name}")

### Prior Predictive Sampling

Before fitting models to data, it's crucial to understand what our priors imply. **Prior predictive sampling** generates data from our model before seeing any observations:

In [ ]:
# Create a simple linear regression model for demonstration
with pm.Model() as linear_model:
    # Priors for regression coefficients
    alpha = pm.Normal('intercept', mu=0, sigma=1)
    beta = pm.Normal('slope', mu=0, sigma=1)
    sigma = pm.HalfNormal('sigma', sigma=1)
    
    # Create some input data
    x_data = np.linspace(-2, 2, 50)
    
    # Define the linear relationship
    mu = pm.Deterministic('mu', alpha + beta * x_data)
    
    # Likelihood (but no observed data yet)
    y = pm.Normal('y', mu=mu, sigma=sigma)
    
    # Sample from the prior predictive distribution
    prior_predictive = pm.sample_prior_predictive(samples=500, random_seed=RANDOM_SEED)

# Visualize prior predictive samples
y_samples = prior_predictive.prior_predictive['y'].values

fig = go.Figure()

# Plot several prior predictive realizations
for i in range(min(20, y_samples.shape[0])):
    fig.add_trace(
        go.Scatter(x=x_data, y=y_samples[i, 0, :],
                  mode='lines', opacity=0.3,
                  line=dict(color='blue'),
                  showlegend=i==0, name='Prior samples')
    )

fig.update_layout(
    title="Prior Predictive Samples from Linear Regression Model",
    xaxis_title="x",
    yaxis_title="y",
    height=500
)

fig.show()

print(f"Prior predictive samples shape: {y_samples.shape}")
print(f"This shows {y_samples.shape[0]} different realizations of our prior beliefs")

### Parameter Constraints and Transformations

Many parameters have natural constraints (e.g., variances must be positive). PyMC automatically handles these constraints through parameter transformations:

In [ ]:
with pm.Model() as constrained_model:
    # Constrained variables
    positive_var = pm.HalfNormal('positive', sigma=1)  # x >= 0
    bounded_var = pm.Beta('bounded', alpha=2, beta=2)  # 0 <= x <= 1
    unrestricted_var = pm.Normal('unrestricted', mu=0, sigma=1)  # x ∈ ℝ
    
    # PyMC automatically creates transformed versions for sampling
    print("Free (transformed) variables for sampling:")
    for rv in constrained_model.free_RVs:
        print(f"  {rv}")
    
    print("\nValue variables (original scale):")
    for rv in constrained_model.value_vars:
        print(f"  {rv}")

**Key Point**: PyMC handles parameter transformations automatically. For example, `HalfNormal` variables are log-transformed during sampling to ensure they remain positive, then back-transformed for interpretation.

---

## Part III: Introduction to PyMC Gaussian Processes

Now that we understand PyMC's fundamentals, let's explore how to work with Gaussian Processes. PyMC provides a comprehensive GP module (`pm.gp`) with implementations optimized for different use cases.

### PyMC's GP Module Structure

PyMC's GP functionality is organized into several key components:

1. **Mean Functions** (`pm.gp.mean`): Define the expected function behavior
2. **Covariance Functions** (`pm.gp.cov`): Define the correlation structure
3. **GP Implementations**: Different computational approaches
   - `pm.gp.Marginal`: Efficient for Gaussian likelihoods
   - `pm.gp.Latent`: Flexible for non-Gaussian likelihoods

### Mean Functions

Mean functions specify the expected value of the GP at each input. Let's explore the built-in options:

In [ ]:
# Create demonstration data
X_demo = np.linspace(0, 10, 100)[:, None]

# Different mean functions
mean_functions = {
    'Zero': pm.gp.mean.Zero(),
    'Constant': pm.gp.mean.Constant(c=2.5),
    'Linear': pm.gp.mean.Linear(coeffs=pt.as_tensor([0.5]), intercept=pt.as_tensor(1.0))
}

# Evaluate mean functions
fig = go.Figure()

colors = ['blue', 'red', 'green']
for (name, mean_func), color in zip(mean_functions.items(), colors):
    mean_values = mean_func(X_demo).eval()
    
    fig.add_trace(
        go.Scatter(x=X_demo.flatten(), y=mean_values,
                  mode='lines', name=f'{name} Mean',
                  line=dict(color=color, width=3))
    )

fig.update_layout(
    title="PyMC Mean Functions",
    xaxis_title="x",
    yaxis_title="Mean function value m(x)",
    height=400
)

fig.show()

print("Mean Functions in PyMC:")
print("• Zero(): m(x) = 0 for all x")
print("• Constant(c): m(x) = c for all x")
print("• Linear(coeffs, intercept): m(x) = intercept + coeffs·x")
print("• And more: Polynomial, custom functions...")

### Covariance Functions (Kernels)

Covariance functions are the heart of GP modeling. They encode our assumptions about function behavior. Let's explore PyMC's built-in kernels:

In [ ]:
# Demonstrate different covariance functions
x_test = np.array([[0.0]])  # Reference point
X_range = np.linspace(-3, 3, 200)[:, None]

# Different covariance functions with similar length scales
kernels = {
    'ExpQuad (RBF)': pm.gp.cov.ExpQuad(1, ls=1.0),
    'Matérn 5/2': pm.gp.cov.Matern52(1, ls=1.0),
    'Matérn 3/2': pm.gp.cov.Matern32(1, ls=1.0),
    'Exponential': pm.gp.cov.Exponential(1, ls=1.0)
}

fig = go.Figure()

colors = ['blue', 'red', 'green', 'orange']
for (name, kernel), color in zip(kernels.items(), colors):
    # Compute covariance with reference point
    cov_values = [kernel(x_test, x).eval().item() for x in X_range]
    
    fig.add_trace(
        go.Scatter(x=X_range.flatten(), y=cov_values,
                  mode='lines', name=name,
                  line=dict(color=color, width=3))
    )

fig.update_layout(
    title="Covariance Functions: k(0, x) vs x",
    xaxis_title="Distance from reference point",
    yaxis_title="Covariance k(0, x)",
    height=500
)

fig.show()

print("Kernel Properties:")
print("• ExpQuad: Infinitely differentiable (very smooth functions)")
print("• Matérn 5/2: Twice differentiable (smooth functions)")
print("• Matérn 3/2: Once differentiable (moderately smooth)")
print("• Exponential: Continuous but not differentiable (rough functions)")

---

## Part IV: GP Implementations in PyMC - A Real Example

Now let's put everything together by building a complete GP regression model using real data. We'll demonstrate both the Marginal and Latent approaches and compare their performance.

### Creating Synthetic Regression Data

Let's create a realistic regression dataset that will showcase the strengths of GP modeling:

In [ ]:
# Generate synthetic regression data with non-linear structure
def true_function(x):
    """A complex non-linear function to learn."""
    return (0.8 * np.sin(2*np.pi*x) + 
            0.3 * np.cos(6*np.pi*x) + 
            0.1 * x**2 - 0.05 * x)

# Training data - deliberately sparse to show GP uncertainty
n_train = 20
X_train = RNG.uniform(0, 1, n_train)[:, None]
X_train = np.sort(X_train, axis=0)

y_true = true_function(X_train.flatten())
noise_std = 0.08
y_train = y_true + RNG.normal(0, noise_std, n_train)

# Test data for predictions
X_test = np.linspace(-0.1, 1.1, 150)[:, None]  # Slightly outside training range
y_test_true = true_function(X_test.flatten())

# Visualize the data
fig = go.Figure()

# True function
fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=y_test_true,
              mode='lines', name='True function',
              line=dict(color='black', width=3, dash='dash'))
)

# Training data
fig.add_trace(
    go.Scatter(x=X_train.flatten(), y=y_train,
              mode='markers', name='Training data',
              marker=dict(color='red', size=10, symbol='circle'))
)

# True (noiseless) training points
fig.add_trace(
    go.Scatter(x=X_train.flatten(), y=y_true,
              mode='markers', name='True (noiseless)',
              marker=dict(color='darkred', size=8, symbol='x'))
)

fig.update_layout(
    title="Synthetic Regression Dataset",
    xaxis_title="x",
    yaxis_title="y",
    height=500,
    legend=dict(x=0.02, y=0.98)
)

fig.show()

print(f"Training data: {n_train} points")
print(f"Noise standard deviation: {noise_std}")
print(f"Training range: [{X_train.min():.2f}, {X_train.max():.2f}]")
print(f"Test range: [{X_test.min():.2f}, {X_test.max():.2f}] (includes extrapolation)")

### Approach 1: Marginal GP

The marginal approach analytically integrates out the latent function, making it computationally efficient for Gaussian likelihoods:

In [ ]:
with pm.Model() as marginal_model:
    
    # Hyperpriors for kernel hyperparameters
    # Length scale: how quickly the covariance decays
    ℓ = pm.InverseGamma("ℓ", alpha=5, beta=5)  # Weakly informative
    
    # Marginal standard deviation: overall function scale
    η = pm.HalfNormal("η", sigma=2)
    
    # Observation noise standard deviation
    σ = pm.HalfNormal("σ", sigma=0.5)
    
    # Mean function (zero for simplicity)
    mean_func = pm.gp.mean.Zero()
    
    # Covariance function: scaled Matérn 5/2 kernel
    cov_func = η**2 * pm.gp.cov.Matern52(1, ℓ)
    
    # GP prior specification
    gp = pm.gp.Marginal(mean_func=mean_func, cov_func=cov_func)
    
    # Marginal likelihood - integrates out the function analytically
    y_obs = gp.marginal_likelihood("y", X=X_train, y=y_train, sigma=σ)
    
    print("Marginal GP Model Structure:")
    print(f"Hyperparameters: {[v.name for v in marginal_model.free_RVs]}")
    print(f"Total free parameters: {len(marginal_model.free_RVs)}")
    print("Note: Latent function values are integrated out analytically")

### Approach 2: Latent GP

The latent approach explicitly includes the function values as parameters, providing more flexibility but at higher computational cost:

In [ ]:
with pm.Model() as latent_model:
    
    # Same hyperpriors
    ℓ = pm.InverseGamma("ℓ", alpha=5, beta=5)
    η = pm.HalfNormal("η", sigma=2)
    σ = pm.HalfNormal("σ", sigma=0.5)
    
    # Same mean and covariance functions
    mean_func = pm.gp.mean.Zero()
    cov_func = η**2 * pm.gp.cov.Matern52(1, ℓ)
    
    # GP specification
    gp = pm.gp.Latent(mean_func=mean_func, cov_func=cov_func)
    
    # Explicit prior over function values at training points
    f = gp.prior("f", X=X_train)
    
    # Likelihood connecting function values to observations
    y_obs = pm.Normal("y", mu=f, sigma=σ, observed=y_train)
    
    print("Latent GP Model Structure:")
    print(f"Hyperparameters: {[v.name for v in latent_model.free_RVs if v.name != 'f']}")
    print(f"Function values: f (dimension {f.eval().shape})")
    print(f"Total free parameters: {len(latent_model.free_RVs)}")
    print("Note: Function values are explicit random variables")

### Model Fitting and Performance Comparison

Let's fit both models and compare their computational performance:

In [ ]:
# Fit the marginal model
print("Fitting Marginal GP model...")
import time
start_time = time.time()

with marginal_model:
    trace_marginal = pm.sample(
        draws=1000,
        tune=1000,
        chains=2,
        target_accept=0.95,
        random_seed=RANDOM_SEED,
        progressbar=False
    )

marginal_time = time.time() - start_time
marginal_ess = az.ess(trace_marginal).min().values

print(f"✓ Marginal model fitted in {marginal_time:.1f}s")
print(f"  Minimum ESS: {marginal_ess:.0f}")
print(f"  ESS per second: {marginal_ess/marginal_time:.1f}")

In [ ]:
# Fit the latent model
print("\nFitting Latent GP model...")
start_time = time.time()

with latent_model:
    trace_latent = pm.sample(
        draws=1000,
        tune=1000,
        chains=2,
        target_accept=0.95,
        random_seed=RANDOM_SEED,
        progressbar=False
    )

latent_time = time.time() - start_time
latent_ess = az.ess(trace_latent, var_names=['ℓ', 'η', 'σ']).min().values

print(f"✓ Latent model fitted in {latent_time:.1f}s")
print(f"  Minimum ESS (hyperparameters): {latent_ess:.0f}")
print(f"  ESS per second: {latent_ess/latent_time:.1f}")
print(f"\nSpeedup factor: {latent_time/marginal_time:.1f}x (Marginal is faster)")

### Generating Predictions

Now let's generate predictions from both models and compare their performance:

In [ ]:
# Generate predictions from both models
print("Generating predictions...")

# Marginal model predictions
with marginal_model:
    f_pred_marginal = gp.conditional("f_pred", X_test)
    pred_marginal = pm.sample_posterior_predictive(
        trace_marginal,
        var_names=["f_pred"],
        progressbar=False,
        random_seed=RANDOM_SEED
    )

# Latent model predictions  
with latent_model:
    f_pred_latent = gp.conditional("f_pred", X_test)
    pred_latent = pm.sample_posterior_predictive(
        trace_latent,
        var_names=["f_pred"],
        progressbar=False,
        random_seed=RANDOM_SEED
    )

# Extract prediction statistics
f_pred_marginal_samples = pred_marginal.posterior_predictive["f_pred"].values
f_pred_mean_marginal = f_pred_marginal_samples.mean(axis=(0, 1))
f_pred_std_marginal = f_pred_marginal_samples.std(axis=(0, 1))

f_pred_latent_samples = pred_latent.posterior_predictive["f_pred"].values
f_pred_mean_latent = f_pred_latent_samples.mean(axis=(0, 1))
f_pred_std_latent = f_pred_latent_samples.std(axis=(0, 1))

print("✓ Predictions generated for both models")

### Results Visualization and Comparison

Let's create a comprehensive comparison of both approaches:

In [ ]:
# Create comprehensive comparison plot
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=["Marginal GP Predictions", "Latent GP Predictions",
                                   "Residuals Comparison", "Uncertainty Comparison"],
                    vertical_spacing=0.1, horizontal_spacing=0.1)

# Function to add GP predictions to subplot
def add_gp_predictions(fig, row, col, X, y_true, y_pred_mean, y_pred_std, 
                      X_train, y_train, color, name_prefix):
    # Confidence interval
    fig.add_trace(
        go.Scatter(
            x=np.concatenate([X.flatten(), X.flatten()[::-1]]),
            y=np.concatenate([y_pred_mean + 2*y_pred_std,
                             (y_pred_mean - 2*y_pred_std)[::-1]]),
            fill='toself',
            fillcolor=f'rgba({"0,100,255" if color=="blue" else "0,200,100"},0.3)',
            line=dict(color='rgba(255,255,255,0)'),
            showlegend=False,
            hoverinfo='skip'
        ),
        row=row, col=col
    )
    
    # Prediction mean
    fig.add_trace(
        go.Scatter(x=X.flatten(), y=y_pred_mean,
                  mode='lines', name=f'{name_prefix} Mean',
                  line=dict(color=color, width=2),
                  showlegend=row==1 and col==1),
        row=row, col=col
    )
    
    # True function
    fig.add_trace(
        go.Scatter(x=X.flatten(), y=y_true,
                  mode='lines', name='True Function',
                  line=dict(color='black', width=2, dash='dash'),
                  showlegend=row==1 and col==1),
        row=row, col=col
    )
    
    # Training data
    fig.add_trace(
        go.Scatter(x=X_train.flatten(), y=y_train,
                  mode='markers', name='Training Data',
                  marker=dict(color='red', size=6),
                  showlegend=row==1 and col==1),
        row=row, col=col
    )

# Add predictions for both models
add_gp_predictions(fig, 1, 1, X_test, y_test_true, f_pred_mean_marginal, 
                  f_pred_std_marginal, X_train, y_train, 'blue', 'Marginal')
add_gp_predictions(fig, 1, 2, X_test, y_test_true, f_pred_mean_latent,
                  f_pred_std_latent, X_train, y_train, 'green', 'Latent')

# Residuals comparison
residuals_marginal = f_pred_mean_marginal - y_test_true
residuals_latent = f_pred_mean_latent - y_test_true

fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=residuals_marginal,
              mode='lines', name='Marginal Residuals',
              line=dict(color='blue', width=2)),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=residuals_latent,
              mode='lines', name='Latent Residuals',
              line=dict(color='green', width=2)),
    row=2, col=1
)
fig.add_hline(y=0, line=dict(color='black', dash='dash'), row=2, col=1)

# Uncertainty comparison
fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=f_pred_std_marginal,
              mode='lines', name='Marginal Std',
              line=dict(color='blue', width=2)),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=X_test.flatten(), y=f_pred_std_latent,
              mode='lines', name='Latent Std', 
              line=dict(color='green', width=2)),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="Comprehensive GP Model Comparison",
    showlegend=True
)

fig.update_xaxes(title_text="x")
fig.update_yaxes(title_text="y", row=1)
fig.update_yaxes(title_text="Residual", row=2, col=1)
fig.update_yaxes(title_text="Standard Deviation", row=2, col=2)

fig.show()

# Performance metrics
mse_marginal = np.mean(residuals_marginal**2)
mse_latent = np.mean(residuals_latent**2)
mae_marginal = np.mean(np.abs(residuals_marginal))
mae_latent = np.mean(np.abs(residuals_latent))

print("\n" + "="*50)
print("PERFORMANCE COMPARISON")
print("="*50)
print(f"Marginal GP:")
print(f"  MSE: {mse_marginal:.6f}")
print(f"  MAE: {mae_marginal:.6f}")
print(f"  Sampling time: {marginal_time:.1f}s")
print(f"\nLatent GP:")
print(f"  MSE: {mse_latent:.6f}")
print(f"  MAE: {mae_latent:.6f}")
print(f"  Sampling time: {latent_time:.1f}s")
print(f"\nDifference:")
print(f"  ΔMSE: {abs(mse_marginal - mse_latent):.6f}")
print(f"  Speed ratio: {latent_time/marginal_time:.1f}x")

### Hyperparameter Posterior Analysis

Let's examine the learned hyperparameters from both models:

In [ ]:
# Compare hyperparameter posteriors
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=["Length Scale (ℓ)", "Marginal Std (η)", "Noise Std (σ)"])

# Extract samples
marginal_samples = az.extract(trace_marginal, num_samples=1000)
latent_samples = az.extract(trace_latent, num_samples=1000, var_names=['ℓ', 'η', 'σ'])

params = ['ℓ', 'η', 'σ']
colors = ['blue', 'green']
names = ['Marginal', 'Latent']

for i, param in enumerate(params):
    # Marginal samples
    fig.add_trace(
        go.Histogram(
            x=marginal_samples[param].values,
            name=names[0] if i == 0 else None,
            opacity=0.7,
            nbinsx=30,
            marker_color=colors[0],
            showlegend=i==0
        ),
        row=1, col=i+1
    )
    
    # Latent samples
    fig.add_trace(
        go.Histogram(
            x=latent_samples[param].values,
            name=names[1] if i == 0 else None,
            opacity=0.7,
            nbinsx=30,
            marker_color=colors[1],
            showlegend=i==0
        ),
        row=1, col=i+1
    )

fig.update_layout(
    height=400,
    title_text="Hyperparameter Posterior Distributions",
    barmode='overlay'
)

fig.update_xaxes(title_text="Parameter value")
fig.update_yaxes(title_text="Frequency")

fig.show()

# Posterior summaries
print("\nHyperparameter Posterior Summaries:")
print("\nMarginal GP:")
for param in params:
    samples = marginal_samples[param].values
    mean_val = samples.mean()
    std_val = samples.std()
    q025, q975 = np.percentile(samples, [2.5, 97.5])
    print(f"  {param}: {mean_val:.3f} ± {std_val:.3f} [{q025:.3f}, {q975:.3f}]")

print("\nLatent GP:")
for param in params:
    samples = latent_samples[param].values
    mean_val = samples.mean()
    std_val = samples.std()
    q025, q975 = np.percentile(samples, [2.5, 97.5])
    print(f"  {param}: {mean_val:.3f} ± {std_val:.3f} [{q025:.3f}, {q975:.3f}]")

---

## Summary: When to Use Which Approach?

Based on our comprehensive comparison, here are the key decision criteria:

### Use **Marginal GP** (`pm.gp.Marginal`) when:
- ✅ **Gaussian likelihood**: You have regression with normal noise
- ✅ **Computational efficiency**: Speed and memory are important
- ✅ **Large datasets**: More than ~100-200 data points
- ✅ **Standard regression**: Basic function interpolation/extrapolation
- ✅ **Production deployment**: Need fast inference

### Use **Latent GP** (`pm.gp.Latent`) when:
- ✅ **Non-Gaussian likelihoods**: Classification, count data, robust regression
- ✅ **Function access needed**: Want posterior samples of the function itself
- ✅ **Complex models**: Hierarchical models, multi-output GPs
- ✅ **Small datasets**: Fewer than ~100 data points
- ✅ **Research/exploration**: Flexibility more important than speed

### Key Takeaways

1. **Performance**: Both approaches yield virtually identical predictions for Gaussian regression
2. **Speed**: Marginal approach is typically 2-5x faster
3. **Memory**: Marginal approach uses less memory (O(n²) vs O(n² + n))
4. **Flexibility**: Latent approach works with any likelihood
5. **Hyperparameters**: Both learn very similar hyperparameter values

### Best Practices

- **Start with Marginal GP** for standard regression problems
- **Use informative priors** on hyperparameters when possible
- **Check prior predictive samples** before fitting
- **Monitor convergence** using effective sample size and R-hat
- **Validate predictions** on held-out test data

---

## Next Steps and Advanced Topics

Congratulations! You've mastered the fundamentals of Gaussian Process modeling with PyMC. You now understand:

✅ **Mathematical foundations**: From multivariate Gaussians to function-space distributions  
✅ **PyMC fundamentals**: Model contexts, distributions, and probabilistic programming  
✅ **Kernel theory**: How covariance functions encode function properties  
✅ **Implementation trade-offs**: When to use marginal vs latent approaches  
✅ **Complete workflow**: From prior specification to posterior analysis  

### Preview of Session 2

In the next session, **"Advanced Kernels and Applications"**, we'll explore:

- **Kernel composition**: Combining kernels for complex patterns
- **Specialized kernels**: Periodic, polynomial, and custom kernels
- **Multi-dimensional inputs**: Handling higher-dimensional data
- **Non-Gaussian likelihoods**: Classification and count data
- **Model selection**: Comparing and validating GP models
- **Scalability**: Techniques for larger datasets

### Practice Exercises

Before the next session, try these exercises:

1. **Experiment with kernels**: Replace Matérn 5/2 with different kernels and observe the changes
2. **Hyperparameter sensitivity**: Vary the prior distributions and see how it affects results
3. **Real data application**: Find a regression dataset and apply GP modeling
4. **Extrapolation analysis**: Evaluate how well GPs extrapolate beyond the training range

### Resources

- **PyMC GP Documentation**: [Official Guide](https://www.pymc.io/projects/docs/en/stable/api/gp.html)
- **Textbook**: Rasmussen & Williams (2006) "Gaussian Processes for Machine Learning" 
- **Examples**: [PyMC GP Gallery](https://www.pymc.io/projects/examples/en/latest/gaussian_processes/index.html)

You've built a solid foundation for advanced GP modeling!